In [44]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [41]:
df = pd.read_excel('Training-testing.xlsx',sheet_name = 'Sheet2')
df.rename(columns = {'Date - Month':'Date'},inplace = True)
df.rename(columns = {'Current Status':'Current_Status'},inplace = True)
df = df.query("Current_Status in ('Active','Resigned','Terminated')")
df.rename(columns = {'Current_Status':'Current Status'},inplace = True)
df

,Date - Year,Date,Ident,Employee Name,Manager,Position,Department,Position Group,Client,Program,Sum of Internal Moves,Market,Sum of Salary,Language,Nationality,Gender.,Current Civil Status,Age,Graduate / Undergrad,University,Major,Location,District,Contract Type,Current Status,Termination Type,Termination Reason,Last Working Day,AHT,Sum of Handled Transactions,Transfers %,Sum of Total Surveys,NPS %,CSAT %,DSAT %,FCR %,QA Monitored,QA Score,"QA Critical Pass % (EUC, BC, CC, ZTP)",QA EUC Pass %,QA BC Pass %,QA CC Pass %,QA ZTP Pass %,Other Monitored Transactions,Coached Calls,Coached Required,Positive Coaching,Coaching Attainment,Avg. Time To Coach Hrs,Sum of Working Hours On Premises,Sum of Working Hours WAHA,Sum of Over-Time Days,Sum of Over-Time Duration,Sum of Working During Holdidays,Utilization %,Productivity %,Occupancy %,ABS %,Sum of Paid Leaves,Sum of Unpaid Leaves,Sum of Sick-Leaves,Sum of No Shows,Sum of Attrition,Approved Actions,Canceled Actions,Approved Green,Canceled Green,Approved Yellow,Canceled Yellow,Approved Orange,Canceled Orange,Approved Red,Canceled Red,Approved Black,Canceled Black,Approved ACM Investigation,Canceled ACM Investigation,Approved Investigation,Canceled Investigation,Address 1
0,2021,December,4394243,Faiza Fili,"Nabil, Ghaith M.",Representative,Representative,Agent,Expedia,Expedia PLAZA New Hire OPM,NaN,Offshore,5500,English,Eritrean,Female,Single,22,Graduate,6Th October Academy,Nursing,TPEG Plaza,haram,Customer Service Representative,Active,NaN,NaN,NaT,NaN,NaN,NaN,6.0,0.500000,0.666667,0.166667,1.000000,4.0,NaN,0.750000,0.750000,1.0,1.0,1.0,16.0,NaN,NaN,NaN,NaN,NaN,185.0,0.0,0.000000,NaN,NaN,0.947898,0.692894,0.730979,0.002613,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12-Caliph-al-Mamoun St.-Al-Haram-Giza
1,2021,December,2631725,Ahmed Nadier Abdelhay,"Abdel, Shimaa H.",Representative,Representative,Agent,Expedia,Expedia PLAZA ECA En Service,NaN,Offshore,5000,English,Egyptian,Male,Single,47,Graduate,-,Accounting,TPEG Plaza,Nozha,Renewal Customer Service Representative,Terminated,Termination for Real and Serious Reason,Found more suitable employment,2021-12-19,369.000000,4.0,NaN,4.0,-1.000000,0.000000,1.000000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.0,206.0,1.000000,NaN,NaN,NaN,NaN,NaN,0.046845,27.0,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12-Ahmed-abdelsalam-zaki-medan-elhegaz-Nozha-C...
2,2021,December,4384330,Omar Mohamed Saleh,"AbdelFattah, Walaa A.",Representative,Representative,Agent,Expedia,Expedia PLAZA Universal Voice,NaN,Offshore,6000,English,Egyptian,Male,Single,23,Graduate,Al-Azhar University,Accounting,TPEG Plaza,15 May City,Customer Service Representative,Terminated,Training,Medical,2021-09-15,261.933333,270.0,NaN,6.0,-0.333333,0.333333,0.666667,0.500000,1.0,NaN,1.000000,1.000000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,206.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,0.046845,27.0,0.0,3.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48-neighborhood 17-fifteenth of may-Cairo
3,2021,December,3040050,Mostafa Gamal Hassan,"Elgazzar, Shady",Representative,Representative,Agent,Expedia,Expedia PLAZA ECA En Service,NaN,Offshore,6000,English,Egyptian,Male,Single,34,Graduate,Ain Shams Uni.,Accounting,TPEG Plaza,Ain Shames,Renewal Customer Service Representative,Terminated,Termination for Real and Serious Reason,Violation of Zero Tolerance Policy,2022-02-14,1200.037129,404.0,NaN,15.0,0.800000,0.866667,0.066667,0.933333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,281.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,0.033808,9.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,12-ain shams-st-elnaam-ain shams-cairo
4,2021,December,3167015,Amr Ali Mahgoub,"Amer, Fatma I.",Representative,Representative,Agent,Expedia,Expedia PLAZA ECA FR Service,NaN,Offshore,6000,French,Egyptian,Male,Single,45,Graduate,Alexandira University,Accounting,TPEG Plaza,awal elra

In [27]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [71]:
def preprocess_input(df):
    df = df.copy()
    # Filtring out null working hours
    df = df.query("Date in ('January' ,'February' , 'December')")
    df = df.fillna(0)
    # droping one value columns or null
    df = df.drop(['Manager','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Department','District','Termination Type','Termination Reason',
                  'Sum of Internal Moves','Transfers %','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %','ABS %','Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation'], axis=1)
    # Binary-encode binary columns
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0,'Never Started':0,'Leave of Absence':0})
    # One-hot encoding
    for column in ['University','Major','Program','Nationality','Current Civil Status']:
        df = onehot_encode(df, column=column)
    
    # Split df into X and y
    y = df['Current Status']
    df = df.reindex(labels=Predict.columns,axis=1)
    df = df.fillna(0)
    X = df
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [68]:
def preprocess_input3(df):
    df = df.copy()
    # Filtring out null working hours
    df = df.query("Date in ('January' ,'February' , 'December')")
    df = df.fillna(0)
    # droping one value columns or null
    df = df.drop(['Manager','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Department','District','Termination Type','Termination Reason',
                  'Sum of Internal Moves','Transfers %','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %','ABS %','Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation'], axis=1)
    # Binary-encode binary columns
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    df['Current Status'] = df['Current Status'].replace({'Active': 0, 'Resigned': 1,'Terminated': 1,'Never Started':0,'Leave of Absence':0})
    # One-hot encoding
    for column in ['University','Major','Program','Current Civil Status']:
        df = onehot_encode(df, column=column)
    
    return df

In [55]:
df2 = preprocess_input3(df)
corr = df2.corr()
corr.head()
corr.to_excel('Correlation.xlsx')

In [72]:
X_train, X_test, y_train, y_test = preprocess_input(df)
X_train

,Date - Year,Date,Sum of Salary,Gender.,Age,Graduate / Undergrad,AHT,Sum of Handled Transactions,Sum of Total Surveys,NPS %,CSAT %,DSAT %,FCR %,QA Monitored,"QA Critical Pass % (EUC, BC, CC, ZTP)",QA EUC Pass %,QA BC Pass %,QA CC Pass %,QA ZTP Pass %,Other Monitored Transactions,Positive Coaching,Sum of Working Hours On Premises,Sum of Working Hours WAHA,Sum of Over-Time Days,Sum of Working During Holdidays,Occupancy %,Sum of Paid Leaves,Sum of Unpaid Leaves,Sum of Sick-Leaves,Sum of No Shows,Sum of Attrition,Approved Actions,Approved Red,Canceled Red,Approved Black,Canceled Black,University_-,University_6Th October Academy,University_Aast,University_Agriculture,University_Ahram Canadian,University_Ain Shams Uni.,University_Akhbar El Youm Academy,University_Al Maarief Higher Institute,University_Al-Azhar University,University_Aleppo Uni.,University_Alexandira University,University_Alson Institue,University_American University In Cairo,University_Aou,University_Arab Academy,University_Arab Open University,University_Art,University_Asd,University_Asia Pacific University,University_Asian Uni In Malayasaia,University_Ast,University_Auc,University_Badr University,University_Baharya Academy,University_Banglor University,University_Benha Uni.,University_British University In Egypt,University_Bue,University_Cac,University_Cairo University,University_Cic,University_Commerce,University_Csc,University_Culture Academy,University_Curtin Uni,University_Cypris International,University_Damanhour Uni.,University_Dropout,University_Eac Uni,University_Ebla Private University,University_El Shrouk Academy,University_El-Ahram Canadian Uni.,University_El-Azhar Uni.,University_El-Fayoum,University_El-Gezira Academy,University_El-Minya Uni.,University_El-Sharqa Uni.,University_El-Shorouk Academy,University_El-Sudan Uni.,University_El-Zagazig Uni.,University_Fayoum Uni.,University_Finance,University_Florida Flights,University_Foreign Trade,University_Fuc,University_Fue,University_Future Academy,University_Galala,University_Grinich Universty,University_Guc,University_Hdi,University_Heliopolis University,University_Helwan University,University_High Institue Of Languages,University_High School,University_High Technological Institute,University_Higher Instite Of Applied Arts (6 Of Octber),University_Higher Institue For Sience And Culture,University_Higher Institue Of Adminstrative Science,University_Higher Institute For Languages And Translation,University_Higher Institution For Technology,University_Iaems,University_Iaet,University_Information System,University_Institut Superieur Des Arts De Guinee,University_Institute Of Reasearches,University_International College Of Language Amd Commerce,University_Kafr Al Sheikh,University_Kazan Petrochemical Uni.,University_Khartoom,University_King Faisl Uni,University_Language German,University_Language Inst,University_Law,University_Madina Academy,University_Madinat Elsadat,University_Mahsa University In Malaysia,University_Malak Hassan El Thany Uni.,University_Mansoura University,University_Markting,University_Masr International University,University_Memorial Uni,University_Minufiya University,University_Misr International University,University_Miu,University_Modern Academy,University_Mokattam Institute,University_Monofeya,University_Msa,University_Mti University,University_Must,University_National School Of Engineering If Tunis,University_New Cairo Academy,University_October 6 University (O6U),University_Others,University_Pacific Technology,University_Port Saied University,University_Riu,University_Robert Gorden Universty,University_Sadaat Academy,University_Sanaa University,University_Science,University_Segi University In Malaysia,University_Sherouk Academy,University_Sohag Uni.,University_South Valley,University_St. Dominic College Of Asia,University_Sudan University,University_Suez Canal University,University_Tabok Uni,University_Tanta University,University_Tartous Uni.,University_Technological College,Univers

In [73]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [74]:
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))

                   Logistic Regression: 88.69%
                   K-Nearest Neighbors: 84.62%
                         Decision Tree: 78.28%
Support Vector Machine (Linear Kernel): 87.33%
   Support Vector Machine (RBF Kernel): 90.95%
                        Neural Network: 90.05%
                         Random Forest: 87.33%
                     Gradient Boosting: 89.14%


In [66]:
def preprocess_input2(df):
    df = df.copy()
    # Filtring out null working hours
    df = df.query("Date == 'March'")
    df = df.fillna(0)
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    #df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0})
    df = df.drop(['Current Status','Manager','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Department','District','Termination Type','Termination Reason',
                  'Sum of Internal Moves','Transfers %','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %','ABS %','Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation'], axis=1)
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    # One-hot encoding
    for column in ['University','Major','Program','Nationality','Current Civil Status']:
        df = onehot_encode(df, column=column)
    
    
    
    return df

In [67]:
Predict = preprocess_input2(df)
df2 = preprocess_input3(df)
Predict.shape

(790, 375)

In [62]:
Predict.columns.difference(df2.columns)

Index(['ABS %', 'Approved ACM Investigation', 'Approved Green',
       'Approved Investigation', 'Approved Orange', 'Approved Yellow',
       'Avg. Time To Coach Hrs', 'Canceled ACM Investigation',
       'Canceled Actions', 'Canceled Green', 'Canceled Investigation',
       'Canceled Orange', 'Canceled Yellow', 'Coached Calls',
       'Coached Required', 'Coaching Attainment', 'Major_Linguistics',
       'Major_Medcine', 'Major_Musician', 'Major_Oriental Languages',
       'Major_Sports Literature', 'Productivity %', 'QA Score',
       'Sum of Internal Moves', 'Sum of Over-Time Duration', 'Transfers %',
       'University_Madinat Elsadat', 'Utilization %'],
      dtype='object')

In [77]:
#Diff_Col = Predict.columns.difference(df2.columns)
#Predict = Predict.drop(columns=Diff_Col)
#Predict = Predict.reindex(labels=df2.columns,axis=1)
#Predict = Predict.drop(columns=['Current Status'], axis = 1)
Predict = Predict.drop(columns=['Test'], axis = 1)
#Predict = Predict.fillna(0)

In [21]:
Neural = MLPClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [78]:
log_mod = LogisticRegression()
log_mod.fit(X_train, y_train)
Predict['Test'] = log_mod.predict(Predict)
df['Outcome'] = Predict['Test']

In [80]:
df.to_excel('Checking.xlsx',index = False)

In [28]:
Predict['Test'].mean()

1.0

In [18]:
df.to_excel('Predicted output.xlsx',index = False)